In [21]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.
# 
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

import os
import numpy as np

# PARAMETERS
# Experiments to load
#xp_name='../failures/inversion_50k'
xp_name='../failures/eigenvectors_50k_90pc'
#xp_name='../failures/eigenvectors_50k_70pc'
#xp_name='../failures/eigenvectors_50k_6x6'

# Task : invert for first XP, eigenvector for the rest
#task="invert"
task="eigenvectors"

# matrix dimension: 6 for the last XP, 5 for the rest
dimension = 5
#dimension = 6

# DECODING THE OUTPUT FILE OF THE MODEL (with verbose=1)

# read a float
def decode_float(s,m,e):
    f=int(m)*10**int(e[1:])
    if s=='-':
        f*=-1
    return f

# read a matrix
def decode_mat(lst):
    dim1 = int(lst[0][1:])
    dim2 = int(lst[1][1:])
    res =np.zeros((dim1,dim2))
    pos=2
    for i in range(dim1):
        for j in range(dim2):
            try:
                res[i][j]=decode_float(lst[pos],lst[pos+1],lst[pos+2]) 
            except Exception as e: return np.ones((dim1,dim2))
            pos+=3 
    return res

# read the file
def read_file(dim, eval_size=50000):
    res=[]
    file = xp_name+'.pred'
    print(file)
    f = open(file,'rt')
    for i in range(eval_size):
        lines=[]
        for j in range(4+dim):
            line = f.readline()[:-1]
            e = line.split('=')
            if len(e)>1:
                lines.append(e[1])
            else:
                lines.append(e[0])
        eq = lines[0]
        l = lines[1]
        for i in range (2,dim+1):
            l += lines[i]
        src = np.matrix(l)
        src=src.reshape((dim,dim))
        tgt = lines[dim+1][1:-1].replace(' ','').replace("'",'').split(',')
        hyp = lines[dim+2][lines[dim+2].index('[')+1:-1].replace(' ','').replace("'",'').split(',')
        tgt = decode_mat(tgt)
        hyp = decode_mat(hyp)
        res.append((src,tgt,hyp))
    return res
    


def check_result(task,m):
    if task=='invert':
        dim = np.shape(m[0])[0]
        p = m[0] @ m[2] - np.eye(dim)
        err = np.sum(np.abs(p)) / dim
        err2 = np.sum(np.abs(m[1]-m[2])) / np.sum(np.abs(m[1]))
        err3=0
        err4 = 0
    elif task == 'eigenvectors':
        s = np.diag(m[2][0])
        dim = np.shape(m[0])[0]
        vec = m[2][1:]
        p = (vec.T @ m[0] @ vec) - s
        err = np.sum(np.abs(p)) / np.sum(np.abs(s))
        
        # eigenvalues
        err2 = np.sum(np.abs(m[1][0]-m[2][0])) / np.sum(np.abs(m[1][0]))
        # eigenvector norms
        err3 = max(np.max(np.abs(np.linalg.norm(m[2][1:],axis=1)-1)),np.max(np.abs(np.linalg.norm(m[2][1:],axis=0)-1)))
        
        # eigenvector perpendicularity        
        # maximum deviation from right angle
        err4 = max(np.max(np.abs([np.dot(m[2][i],m[2][i+1])/(np.linalg.norm(m[2][i])*np.linalg.norm(m[2][i+1])) for i in range(1,dim)])),np.abs(np.dot(m[2][1],m[2][dim]))/(np.linalg.norm(m[2][1])*np.linalg.norm(m[2][dim])),
                  np.max(np.abs([np.dot(m[2][1:][i],m[2][1:][i+1])/(np.linalg.norm(m[2][1:][i])*np.linalg.norm(m[2][1:][i+1])) for i in range(0,dim-1)])),
                         np.abs(np.dot(m[2][1:][0],m[2][1:][dim-1])))
        
    return err,err2,err3, err4


# main code 

mats=read_file(dimension)
tolerances=[0.005,0.01, 0.02,0.05,0.075,0.1,0.15,0.2,0.25]
errors=[0,0,0,0,0,0,0,0,0]
errors2=[0,0,0,0,0,0,0,0,0]
errors3=[0,0,0,0,0,0,0,0,0]
errors4=[0,0,0,0,0,0,0,0,0]

goodcond=[]
badcond=[]
verybadcond=[]
verygoodcond=[]

for m in mats:
    err,err2,err3,err4 = check_result(task,m)
    if task =='invert':
        c=np.linalg.cond(m[0])
    else:
        c=np.linalg.cond(m[2][1:])
        
    if not np.isinf(c):
        if err<0.05:
            goodcond.append(c)
        else:
            badcond.append(c)
        if err >=0.01:
            verybadcond.append(c)
        else: 
            verygoodcond.append(c)

    for i,tol in enumerate(tols):
        if err < tol: errors[i]+=1
        if err2 < tol: errors2[i]+=1
        if err3 < tol: errors3[i]+=1
        if err4 < tol: errors4[i]+=1

print("accuracy")
print("tolerance: % correct")
for i,t in enumerate(tols):
    print(t,":",errors[i]/500)
print("")
if task == 'invert':
    print("approximation of inverse")
    print("tolerance: % correct")
    for i,t in enumerate(tols):
        print(t,":",errors2[i]/500)
    print("")

else:
    print("eigenvalues")
    print("tolerance: % correct")
    for i,t in enumerate(tols):
        print(t,":",errors2[i]/500)
    print("")

    print("eigenvector norms")
    print("tolerance: % correct")
    for i,t in enumerate(tols):
        print(t,":",errors3[i]/500)
    print("")
    print("eigenvector orthogonality")
    print("tolerance: % correct")
    for i,t in enumerate(tols):
        print(t,":",errors4[i]/500)
    print("")

print("5% tolerance: condition numbers")
print("correct predictions:")
print("mean:",np.mean(goodcond))
print("98 percentile:",np.quantile(goodcond,0.98))
print("max:", max(goodcond))
print("stddev:",np.std(goodcond))
print("# of cases:",len(goodcond))
print()
print("incorrect predictions:")
print("mean:",np.mean(badcond))
print("2 percentile:",np.quantile(badcond,0.02))
print("min:",min(badcond))
print("std dev:",np.std(badcond))
print("# of cases:",len(badcond))
print("")
print()
print("1% tolerance: condition numbers")
print("correct predictions:")
print("mean:",np.mean(verygoodcond))
print("98 percentile:",np.quantile(verygoodcond,0.98))
print("max:",max(verygoodcond))
print("stddev:",np.std(verygoodcond))
print("# of cases:",len(verygoodcond))
print()
print("incorrect predictions:")
print("mean:",np.mean(verybadcond))
print("2 percentile:",np.quantile(verybadcond,0.02))
print("min:",min(verybadcond))
print("std dev:",np.std(verybadcond))
print("# of cases:",len(verybadcond))

../failures/eigenvectors_50k_90pc.pred
accuracy
tolerance: % correct
0.005 : 0.012
0.01 : 9.194
0.02 : 76.25
0.05 : 91.96
0.075 : 93.366
0.1 : 94.07
0.15 : 94.896
0.2 : 95.454
0.25 : 95.84

eigenvalues
tolerance: % correct
0.005 : 96.132
0.01 : 99.944
0.02 : 99.994
0.05 : 99.994
0.075 : 99.996
0.1 : 99.996
0.15 : 99.996
0.2 : 99.996
0.25 : 99.996

eigenvector norms
tolerance: % correct
0.005 : 94.644
0.01 : 98.864
0.02 : 99.61
0.05 : 99.88
0.075 : 99.946
0.1 : 99.964
0.15 : 99.98
0.2 : 99.986
0.25 : 99.992

eigenvector orthogonality
tolerance: % correct
0.005 : 51.666
0.01 : 83.922
0.02 : 90.75
0.05 : 93.616
0.075 : 94.554
0.1 : 95.15
0.15 : 95.878
0.2 : 96.42
0.25 : 96.84

5% tolerance: condition numbers
correct predictions:
mean: 1.0102350851244326
98 percentile: 1.0311592627928396
max: 1.109708082809955
stddev: 0.006478954683710473
# of cases: 45980

incorrect predictions:
mean: 1.2849138960821085e+108
2 percentile: 1.042245162084653
min: 1.0202477718165408
std dev: 8.14578661153492